# Imports

In [50]:
import datetime
import logging

import pandas as pd

import core.finance.bid_ask as cfibiask
import im_v2.common.data.transform.transform_utils as imvcdttrut
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import helpers.hdatetime as hdateti
import helpers.hparquet as hparque
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc

In [51]:
hdbg.init_logger(verbosity=logging.INFO)
log_level = logging.INFO

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.5.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmampTask4827_Resample_200ms_Binance_bid_ask_data_to_1_minute'
  hash='3c0875f4f'
  # Last commits:
    * 3c0875f4f sonaalKant Debug mismatch and fix sampling                                   (    3 days ago) Fri Aug 4 21:49:55 2023  (HEAD -> CmampTask4827_Resample_200ms_Binance_bid_ask_data_to_1_minute, origin/CmampTask4827_Resample_200ms_Binance_bid_ask_data_to_1_minute)
    * 82d532666 jsmerix  Add a TODO for Sonaal                                             (    4 days ago) Thu Aug 3 16:17:25 2023           
    * 2a54111f0 jsmerix  Checkpoint                              

## Set params

In [52]:
start = hdateti.convert_timestamp_to_unix_epoch(
    pd.Timestamp("2023-05-05 14:00:00+00:00"), unit="ms"
)
end = hdateti.convert_timestamp_to_unix_epoch(
    pd.Timestamp("2023-05-05 14:10:00+00:00"), unit="ms"
)
s3_path = "s3://cryptokaizen-data.preprod/v3/periodic_daily/airflow/archived_200ms/parquet/bid_ask/futures/v7/ccxt/binance/v1_0_0/"

In [53]:
filters = [("year", "=", 2023), ("month", "=", 5), ("day", "=", 5), 
           ("timestamp", ">=", start), ("timestamp", "<=", end), ("currency_pair", "=", "BTC_USDT")
          ]
data = hparque.from_parquet(
        "s3://cryptokaizen-data.preprod/v3/periodic_daily/airflow/archived_200ms/parquet/bid_ask/futures/v7/ccxt/binance/v1_0_0/", filters=filters, aws_profile="ck"
)

## Drop duplicates

In [54]:
data = data.drop_duplicates(
    subset=["timestamp", "exchange_id", "currency_pair", "level"]
)
# Timestamp is in the index, column is obsolete
data = data.drop("timestamp", axis=1)

## Transform from long to wide format

In [55]:
data = cfibiask.transform_bid_ask_long_data_to_wide(data, "timestamp")

## Resample

### Resample using approach of double resampling (first to 1 sec, then to 1 min)

In [56]:
## The function name is not 100% correct since the input data need not be 1sec.
data_resampled = imvcdttrut.resample_multilevel_bid_ask_data_from_1sec_to_1min(data)

In [57]:
data_resampled.head()

,level_1.bid_price.close,level_1.bid_size.close,level_1.ask_price.close,level_1.ask_size.close,level_1.bid_price.high,level_1.bid_size.max,level_1.ask_price.high,level_1.ask_size.max,level_1.bid_price.low,level_1.bid_size.min,level_1.ask_price.low,level_1.ask_size.min,level_1.bid_price.mean,level_1.bid_size.mean,level_1.ask_price.mean,level_1.ask_size.mean,level_2.bid_price.close,level_2.bid_size.close,level_2.ask_price.close,level_2.ask_size.close,level_2.bid_price.high,level_2.bid_size.max,level_2.ask_price.high,level_2.ask_size.max,level_2.bid_price.low,level_2.bid_size.min,level_2.ask_price.low,level_2.ask_size.min,level_2.bid_price.mean,level_2.bid_size.mean,level_2.ask_price.mean,level_2.ask_size.mean,level_3.bid_price.close,level_3.bid_size.close,level_3.ask_price.close,level_3.ask_size.close,level_3.bid_price.high,level_3.bid_size.max,level_3.ask_price.high,level_3.ask_size.max,level_3.bid_price.low,level_3.bid_size.min,level_3.ask_price.low,level_3.ask_size.min,level_3.bid_price.mean,level_3.bid_size.mean,level_3.ask_price.mean,level_3.ask_size.mean,level_4.bid_price.close,level_4.bid_size.close,level_4.ask_price.close,level_4.ask_size.close,level_4.bid_price.high,level_4.bid_size.max,level_4.ask_price.high,level_4.ask_size.max,level_4.bid_price.low,level_4.bid_size.min,level_4.ask_price.low,level_4.ask_size.min,level_4.bid_price.mean,level_4.bid_size.mean,level_4.ask_price.mean,level_4.ask_size.mean,level_5.bid_price.close,level_5.bid_size.close,level_5.ask_price.close,level_5.ask_size.close,level_5.bid_price.high,level_5.bid_size.max,level_5.ask_price.high,level_5.ask_size.max,level_5.bid_price.low,level_5.bid_size.min,level_5.ask_price.low,level_5.ask_size.min,level_5.bid_price.mean,level_5.bid_size.mean,level_5.ask_price.mean,level_5.ask_size.mean,level_6.bid_price.close,level_6.bid_size.close,level_6.ask_price.close,level_6.ask_size.close,level_6.bid_price.high,level_6.bid_size.max,level_6.ask_price.high,level_6.ask_size.max,level_6.bid_price.low,level_6.bid_size.min,level_6.ask_price.low,level_6.ask_size.min,level_6.bid_price.mean,level_6.bid_size.mean,level_6.ask_price.mean,level_6.ask_size.mean,level_7.bid_price.close,level_7.bid_size.close,level_7.ask_price.close,level_7.ask_size.close,level_7.bid_price.high,level_7.bid_size.max,level_7.ask_price.high,level_7.ask_size.max,level_7.bid_price.low,level_7.bid_size.min,level_7.ask_price.low,level_7.ask_size.min,level_7.bid_price.mean,level_7.bid_size.mean,level_7.ask_price.mean,level_7.ask_size.mean,level_8.bid_price.close,level_8.bid_size.close,level_8.ask_price.close,level_8.ask_size.close,level_8.bid_price.high,level_8.bid_size.max,level_8.ask_price.high,level_8.ask_size.max,level_8.bid_price.low,level_8.bid_size.min,level_8.ask_price.low,level_8.ask_size.min,level_8.bid_price.mean,level_8.bid_size.mean,level_8.ask_price.mean,level_8.ask_size.mean,level_9.bid_price.close,level_9.bid_size.close,level_9.ask_price.close,level_9.ask_size.close,level_9.bid_price.high,level_9.bid_size.max,level_9.ask_price.high,level_9.ask_size.max,level_9.bid_price.low,level_9.bid_size.min,level_9.ask_price.low,level_9.ask_size.min,level_9.bid_price.mean,level_9.bid_size.mean,level_9.ask_price.mean,level_9.ask_size.mean,level_10.bid_price.close,level_10.bid_size.close,level_10.ask_price.close,level_10.ask_size.close,level_10.bid_price.high,level_10.bid_size.max,level_10.ask_price.high,level_10.ask_size.max,level_10.bid_price.low,level_10.bid_size.min,level_10.ask_price.low,level_10.ask_size.min,level_10.bid_price.mean,level_10.bid_size.mean,level_10.ask_price.mean,level_10.ask_size.mean,exchange_id
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-05 14:01:00+00:00,29177.600000,8.910250,29177.700000,5.501500,29187.70,31.5235,29187.80,23.995400,29161.575,0.176000,29161.70,0.271000,29179.158722,6.285309,29179.267389,7.156755,29177.500000,0.001000,29177.80000

### Resample using approach of single resampling (fdirectly 200 ms to 1 min)

In [58]:
data_resampled2 = imvcdttrut.resample_multilevel_bid_ask_data_to_1min(data)

In [59]:
data_resampled2.head()

,level_1.bid_price.close,level_1.bid_size.close,level_1.ask_price.close,level_1.ask_size.close,level_1.bid_price.high,level_1.bid_size.max,level_1.ask_price.high,level_1.ask_size.max,level_1.bid_price.low,level_1.bid_size.min,level_1.ask_price.low,level_1.ask_size.min,level_1.bid_price.mean,level_1.bid_size.mean,level_1.ask_price.mean,level_1.ask_size.mean,level_2.bid_price.close,level_2.bid_size.close,level_2.ask_price.close,level_2.ask_size.close,level_2.bid_price.high,level_2.bid_size.max,level_2.ask_price.high,level_2.ask_size.max,level_2.bid_price.low,level_2.bid_size.min,level_2.ask_price.low,level_2.ask_size.min,level_2.bid_price.mean,level_2.bid_size.mean,level_2.ask_price.mean,level_2.ask_size.mean,level_3.bid_price.close,level_3.bid_size.close,level_3.ask_price.close,level_3.ask_size.close,level_3.bid_price.high,level_3.bid_size.max,level_3.ask_price.high,level_3.ask_size.max,level_3.bid_price.low,level_3.bid_size.min,level_3.ask_price.low,level_3.ask_size.min,level_3.bid_price.mean,level_3.bid_size.mean,level_3.ask_price.mean,level_3.ask_size.mean,level_4.bid_price.close,level_4.bid_size.close,level_4.ask_price.close,level_4.ask_size.close,level_4.bid_price.high,level_4.bid_size.max,level_4.ask_price.high,level_4.ask_size.max,level_4.bid_price.low,level_4.bid_size.min,level_4.ask_price.low,level_4.ask_size.min,level_4.bid_price.mean,level_4.bid_size.mean,level_4.ask_price.mean,level_4.ask_size.mean,level_5.bid_price.close,level_5.bid_size.close,level_5.ask_price.close,level_5.ask_size.close,level_5.bid_price.high,level_5.bid_size.max,level_5.ask_price.high,level_5.ask_size.max,level_5.bid_price.low,level_5.bid_size.min,level_5.ask_price.low,level_5.ask_size.min,level_5.bid_price.mean,level_5.bid_size.mean,level_5.ask_price.mean,level_5.ask_size.mean,level_6.bid_price.close,level_6.bid_size.close,level_6.ask_price.close,level_6.ask_size.close,level_6.bid_price.high,level_6.bid_size.max,level_6.ask_price.high,level_6.ask_size.max,level_6.bid_price.low,level_6.bid_size.min,level_6.ask_price.low,level_6.ask_size.min,level_6.bid_price.mean,level_6.bid_size.mean,level_6.ask_price.mean,level_6.ask_size.mean,level_7.bid_price.close,level_7.bid_size.close,level_7.ask_price.close,level_7.ask_size.close,level_7.bid_price.high,level_7.bid_size.max,level_7.ask_price.high,level_7.ask_size.max,level_7.bid_price.low,level_7.bid_size.min,level_7.ask_price.low,level_7.ask_size.min,level_7.bid_price.mean,level_7.bid_size.mean,level_7.ask_price.mean,level_7.ask_size.mean,level_8.bid_price.close,level_8.bid_size.close,level_8.ask_price.close,level_8.ask_size.close,level_8.bid_price.high,level_8.bid_size.max,level_8.ask_price.high,level_8.ask_size.max,level_8.bid_price.low,level_8.bid_size.min,level_8.ask_price.low,level_8.ask_size.min,level_8.bid_price.mean,level_8.bid_size.mean,level_8.ask_price.mean,level_8.ask_size.mean,level_9.bid_price.close,level_9.bid_size.close,level_9.ask_price.close,level_9.ask_size.close,level_9.bid_price.high,level_9.bid_size.max,level_9.ask_price.high,level_9.ask_size.max,level_9.bid_price.low,level_9.bid_size.min,level_9.ask_price.low,level_9.ask_size.min,level_9.bid_price.mean,level_9.bid_size.mean,level_9.ask_price.mean,level_9.ask_size.mean,level_10.bid_price.close,level_10.bid_size.close,level_10.ask_price.close,level_10.ask_size.close,level_10.bid_price.high,level_10.bid_size.max,level_10.ask_price.high,level_10.ask_size.max,level_10.bid_price.low,level_10.bid_size.min,level_10.ask_price.low,level_10.ask_size.min,level_10.bid_price.mean,level_10.bid_size.mean,level_10.ask_price.mean,level_10.ask_size.mean,exchange_id
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-05 14:01:00+00:00,29177.6,8.586,29177.7,6.646,29187.7,51.558,29187.8,30.210,29161.3,0.001,29161.4,0.001,29179.209426,5.945873,29179.319672,7.464578,29177.5,0.001,29177.8,0.001,29187.6,23.685,29187.9,19.667,29161.

In [60]:
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import pandas as pd
signature = "periodic_daily.airflow.resampled_1min.parquet.bid_ask.futures.v7.ccxt.binance.v1_0_0"
reader = imvcdcimrdc.RawDataReader(signature, stage="test")
data2 = reader.read_data(pd.Timestamp("2023-05-05T00:00:00+00:00"), pd.Timestamp("2023-05-06T00:00:00+00:00"))

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


TODO(Sonaal):

In the two cells below you see we have run into a discrepancy:
1. The first cell shows what a "raw" bid/ask data we collect look like (sampled ~200ms), notice this row:

timestamp bid_size_l1 	bid_size_l2 	ask_size_l1 	ask_size_l2

2023-05-05 14:04:59.958000+00:00 	7.644 	3.794 	2.962 	0.002

- this is the last row in the minute interval <14:04:00, 14:05:00) and bid_size = 7.644
- the problem is that the resampled data:

timestamp level_1.bid_price.close 	level_1.bid_size.close

2023-05-05 14:05:00+00:00 	29156.700 	8.526667 

show that level_1.bid_size.close = 8.526667 while it should be 7.644

Suggested debugging steps:
1. Firstly let's try to resample the raw data in a new cell using `resample_multilevel_bid_ask_data_to_1min` from `im_v2.common.data.transform.transform_utils` and observe if the same issue occurs, if yes, that means there is a problem within the function.
2. If the function seems to provide the correct values, let's take a look at `im_v2/common/data/transform/resample_daily_bid_ask_data.py` which is a script that produced the incorrect data - it does some transformation before applying the above mentioned resampling function - that would indicate that somewhere in the process there is a bug.


@Juraj I guess I figured out what was the issue causing the discrepancy in the results.
this is the last row in the minute interval <14:04:00, 14:05:00) and bid_size = 7.644
```
timestamp bid_size_l1 bid_size_l2 ask_size_l1 ask_size_l2
2023-05-05 14:04:59.958000+00:00 7.644 3.794 2.962 0.002
```
the problem is that the resampled data:
```
timestamp level_1.bid_price.close level_1.bid_size.close
2023-05-05 14:05:00+00:00 29156.700 8.526667
```
Why level_1.bid_size.close = 8.526667 while it should be 7.644 ?

This is caused by the method ```resample_multilevel_bid_ask_data_to_1min ``` which internally calls the method ```resample_bid_ask_data_from_1sec_to_1min ```. This method first resample the data for ```1sec``` using the config
```
# First resample to 1 second.
    resampling_groups_1sec = [
        (
            {
                "bid_price": "bid_price",
                "bid_size": "bid_size",
                "ask_price": "ask_price",
                "ask_size": "ask_size",
            },
            "mean",
            {},
        ),
    ]
```
This config takes the mean of the value of the values in 1sec interval.
Eg:
```


2023-05-05 14:04:59.043000+00:00 | 9.943 | 3.825 | 5.014 | 0.002
-- | -- | -- | -- | --
7.993 | 3.825 | 5.087 | 0.002
7.644 | 3.794 | 2.962 | 0.002

```
This is the last 1sec data. If you take the mean of bid ask in this 1sec interval ```(9.943 + 7.993 + 7.644 ) / 3 = 8.52667 ``` which is what we are getting as output. I hope this is what you were trying to debug for 1.
Let me know if you want me to fix it or you were looking for something else in debugging.

In [61]:
data.loc["2023-05-05 14:03:59":].head(5 * 60 + 10)[["bid_size_l1", "bid_size_l2", "ask_size_l1", "ask_size_l2"]]

,bid_size_l1,bid_size_l2,ask_size_l1,ask_size_l2
timestamp,,,,
2023-05-05 14:03:59.018000+00:00,3.994,0.001,9.166,0.004
2023-05-05 14:03:59.220000+00:00,4.004,0.001,8.301,0.004
2023-05-05 14:03:59.423000+00:00,2.770,0.001,8.097,0.004
2023-05-05 14:03:59.626000+00:00,3.711,0.001,8.075,0.004
2023-05-05 14:03:59.827000+00:00,3.509,0.655,8.149,1.004
2023-05-05 14:04:00.029000+00:00,2.587,0.655,7.663,1.004
2023-05-05 14:04:00.230000+00:00,2.811,0.655,7.563,1.004
2023-05-05 14:04:00.433000+00:00,4.910,0.655,7.234,1.004
2023-05-05 14:04:00.938000+00:00,17.007,4.420,2.268,0.054


In [62]:
data2.loc["2023-05-05 14:05:00":]

,timestamp,level_1.bid_price.close,level_1.bid_size.close,level_1.ask_price.close,level_1.ask_size.close,level_1.bid_price.high,level_1.bid_size.max,level_1.ask_price.high,level_1.ask_size.max,level_1.bid_price.low,level_1.bid_size.min,level_1.ask_price.low,level_1.ask_size.min,level_1.bid_price.mean,level_1.bid_size.mean,level_1.ask_price.mean,level_1.ask_size.mean,level_2.bid_price.close,level_2.bid_size.close,level_2.ask_price.close,level_2.ask_size.close,level_2.bid_price.high,level_2.bid_size.max,level_2.ask_price.high,level_2.ask_size.max,level_2.bid_price.low,level_2.bid_size.min,level_2.ask_price.low,level_2.ask_size.min,level_2.bid_price.mean,level_2.bid_size.mean,level_2.ask_price.mean,level_2.ask_size.mean,level_3.bid_price.close,level_3.bid_size.close,level_3.ask_price.close,level_3.ask_size.close,level_3.bid_price.high,level_3.bid_size.max,level_3.ask_price.high,level_3.ask_size.max,level_3.bid_price.low,level_3.bid_size.min,level_3.ask_price.low,level_3.ask_size.min,level_3.bid_price.mean,level_3.bid_size.mean,level_3.ask_price.mean,level_3.ask_size.mean,level_4.bid_price.close,level_4.bid_size.close,level_4.ask_price.close,level_4.ask_size.close,level_4.bid_price.high,level_4.bid_size.max,level_4.ask_price.high,level_4.ask_size.max,level_4.bid_price.low,level_4.bid_size.min,level_4.ask_price.low,level_4.ask_size.min,level_4.bid_price.mean,level_4.bid_size.mean,level_4.ask_price.mean,level_4.ask_size.mean,level_5.bid_price.close,level_5.bid_size.close,level_5.ask_price.close,level_5.ask_size.close,level_5.bid_price.high,level_5.bid_size.max,level_5.ask_price.high,level_5.ask_size.max,level_5.bid_price.low,level_5.bid_size.min,level_5.ask_price.low,level_5.ask_size.min,level_5.bid_price.mean,level_5.bid_size.mean,level_5.ask_price.mean,level_5.ask_size.mean,level_6.bid_price.close,level_6.bid_size.close,level_6.ask_price.close,level_6.ask_size.close,level_6.bid_price.high,level_6.bid_size.max,level_6.ask_price.high,level_6.ask_size.max,level_6.bid_price.low,level_6.bid_size.min,level_6.ask_price.low,level_6.ask_size.min,level_6.bid_price.mean,level_6.bid_size.mean,level_6.ask_price.mean,level_6.ask_size.mean,level_7.bid_price.close,level_7.bid_size.close,level_7.ask_price.close,level_7.ask_size.close,level_7.bid_price.high,level_7.bid_size.max,level_7.ask_price.high,level_7.ask_size.max,level_7.bid_price.low,level_7.bid_size.min,level_7.ask_price.low,level_7.ask_size.min,level_7.bid_price.mean,level_7.bid_size.mean,level_7.ask_price.mean,level_7.ask_size.mean,level_8.bid_price.close,level_8.bid_size.close,level_8.ask_price.close,level_8.ask_size.close,level_8.bid_price.high,level_8.bid_size.max,level_8.ask_price.high,level_8.ask_size.max,level_8.bid_price.low,level_8.bid_size.min,level_8.ask_price.low,level_8.ask_size.min,level_8.bid_price.mean,level_8.bid_size.mean,level_8.ask_price.mean,level_8.ask_size.mean,level_9.bid_price.close,level_9.bid_size.close,level_9.ask_price.close,level_9.ask_size.close,level_9.bid_price.high,level_9.bid_size.max,level_9.ask_price.high,level_9.ask_size.max,level_9.bid_price.low,level_9.bid_size.min,level_9.ask_price.low,level_9.ask_size.min,level_9.bid_price.mean,level_9.bid_size.mean,level_9.ask_price.mean,level_9.ask_size.mean,level_10.bid_price.close,level_10.bid_size.close,level_10.ask_price.close,level_10.ask_size.close,level_10.bid_price.high,level_10.bid_size.max,level_10.ask_price.high,level_10.ask_size.max,level_10.bid_price.low,level_10.bid_size.min,level_10.ask_price.low,level_10.ask_size.min,level_10.bid_price.mean,level_10.bid_size.mean,level_10.ask_price.mean,level_10.ask_size.mean,exchange_id,knowledge_timestamp,currency_pair,year,month
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-05 14:05:00+00:00,1683295500000,29156.700,8.526667,29156.800,4.354333,29176.10,25.87100,29176.20,16.392400,29150.8600,0.311000,29150.9600,0.299750,29163

In [63]:
data_resampled3 = imvcdttrut.resample_multilevel_bid_ask_data_to_1min(data)

In [64]:
data_resampled3.loc["2023-05-05 14:05:00":]

,level_1.bid_price.close,level_1.bid_size.close,level_1.ask_price.close,level_1.ask_size.close,level_1.bid_price.high,level_1.bid_size.max,level_1.ask_price.high,level_1.ask_size.max,level_1.bid_price.low,level_1.bid_size.min,level_1.ask_price.low,level_1.ask_size.min,level_1.bid_price.mean,level_1.bid_size.mean,level_1.ask_price.mean,level_1.ask_size.mean,level_2.bid_price.close,level_2.bid_size.close,level_2.ask_price.close,level_2.ask_size.close,level_2.bid_price.high,level_2.bid_size.max,level_2.ask_price.high,level_2.ask_size.max,level_2.bid_price.low,level_2.bid_size.min,level_2.ask_price.low,level_2.ask_size.min,level_2.bid_price.mean,level_2.bid_size.mean,level_2.ask_price.mean,level_2.ask_size.mean,level_3.bid_price.close,level_3.bid_size.close,level_3.ask_price.close,level_3.ask_size.close,level_3.bid_price.high,level_3.bid_size.max,level_3.ask_price.high,level_3.ask_size.max,level_3.bid_price.low,level_3.bid_size.min,level_3.ask_price.low,level_3.ask_size.min,level_3.bid_price.mean,level_3.bid_size.mean,level_3.ask_price.mean,level_3.ask_size.mean,level_4.bid_price.close,level_4.bid_size.close,level_4.ask_price.close,level_4.ask_size.close,level_4.bid_price.high,level_4.bid_size.max,level_4.ask_price.high,level_4.ask_size.max,level_4.bid_price.low,level_4.bid_size.min,level_4.ask_price.low,level_4.ask_size.min,level_4.bid_price.mean,level_4.bid_size.mean,level_4.ask_price.mean,level_4.ask_size.mean,level_5.bid_price.close,level_5.bid_size.close,level_5.ask_price.close,level_5.ask_size.close,level_5.bid_price.high,level_5.bid_size.max,level_5.ask_price.high,level_5.ask_size.max,level_5.bid_price.low,level_5.bid_size.min,level_5.ask_price.low,level_5.ask_size.min,level_5.bid_price.mean,level_5.bid_size.mean,level_5.ask_price.mean,level_5.ask_size.mean,level_6.bid_price.close,level_6.bid_size.close,level_6.ask_price.close,level_6.ask_size.close,level_6.bid_price.high,level_6.bid_size.max,level_6.ask_price.high,level_6.ask_size.max,level_6.bid_price.low,level_6.bid_size.min,level_6.ask_price.low,level_6.ask_size.min,level_6.bid_price.mean,level_6.bid_size.mean,level_6.ask_price.mean,level_6.ask_size.mean,level_7.bid_price.close,level_7.bid_size.close,level_7.ask_price.close,level_7.ask_size.close,level_7.bid_price.high,level_7.bid_size.max,level_7.ask_price.high,level_7.ask_size.max,level_7.bid_price.low,level_7.bid_size.min,level_7.ask_price.low,level_7.ask_size.min,level_7.bid_price.mean,level_7.bid_size.mean,level_7.ask_price.mean,level_7.ask_size.mean,level_8.bid_price.close,level_8.bid_size.close,level_8.ask_price.close,level_8.ask_size.close,level_8.bid_price.high,level_8.bid_size.max,level_8.ask_price.high,level_8.ask_size.max,level_8.bid_price.low,level_8.bid_size.min,level_8.ask_price.low,level_8.ask_size.min,level_8.bid_price.mean,level_8.bid_size.mean,level_8.ask_price.mean,level_8.ask_size.mean,level_9.bid_price.close,level_9.bid_size.close,level_9.ask_price.close,level_9.ask_size.close,level_9.bid_price.high,level_9.bid_size.max,level_9.ask_price.high,level_9.ask_size.max,level_9.bid_price.low,level_9.bid_size.min,level_9.ask_price.low,level_9.ask_size.min,level_9.bid_price.mean,level_9.bid_size.mean,level_9.ask_price.mean,level_9.ask_size.mean,level_10.bid_price.close,level_10.bid_size.close,level_10.ask_price.close,level_10.ask_size.close,level_10.bid_price.high,level_10.bid_size.max,level_10.ask_price.high,level_10.ask_size.max,level_10.bid_price.low,level_10.bid_size.min,level_10.ask_price.low,level_10.ask_size.min,level_10.bid_price.mean,level_10.bid_size.mean,level_10.ask_price.mean,level_10.ask_size.mean,exchange_id
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-05 14:05:00+00:00,29156.7,7.644,29156.8,2.962,29177.9,29.923,29178.0,29.181,29150.0,0.001,29150.1,0.001,29163.052697,5.929726,29163.157261,6.842822,29156.6,3.794,29156.9,0.002,29177.8,13.393,29178.6,14.096,29149.

Now level_1.bid_size.close is consistent with the data. 


In [291]:
filters = [("year", "=", 2023), ("month", "=", 5), ("day", "=", 5), 
           ("timestamp", ">=", start), ("timestamp", "<=", end), ("currency_pair", "in", ["BTC_USDT", "ETH_USDT"])
          ]
data = hparque.from_parquet(
        "s3://cryptokaizen-data.preprod/v3/periodic_daily/airflow/archived_200ms/parquet/bid_ask/futures/v7/ccxt/binance/v1_0_0/", filters=filters, aws_profile="ck"
)

In [292]:
data = data[(data["level"] == 1) | (data["level"] == 2)]
all_cols = imvcdttrut.BID_ASK_COLS + ["timestamp", "currency_pair", "level", "exchange_id"]
data = data[all_cols]
sample_long = data.loc["2023-05-05 14:00:00": "2023-05-05 14:00:01"]

/tmp/ipykernel_667/1022383583.py:4: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  sample_long = data.loc["2023-05-05 14:00:00": "2023-05-05 14:00:01"]


In [293]:
sample_long.head()
# sample_long.to_csv("../data/transform/test/outcomes/SampleBidAskDataLong.csv")

,bid_price,bid_size,ask_price,ask_size,timestamp,currency_pair,level,exchange_id
timestamp,,,,,,,,
2023-05-05 14:00:00.094000+00:00,29163.7,8.581,29163.8,1.762,1683295200094,BTC_USDT,1,binance
2023-05-05 14:00:00.094000+00:00,29163.6,0.004,29163.9,0.177,1683295200094,BTC_USDT,2,binance
2023-05-05 14:00:00.294000+00:00,29165.3,5.552,29165.4,2.276,1683295200294,BTC_USDT,1,binance
2023-05-05 14:00:00.294000+00:00,29165.2,4.331,29165.5,0.533,1683295200294,BTC_USDT,2,binance
2023-05-05 14:00:00.494000+00:00,29165.8,11.740,29166.0,0.002,1683295200494,BTC_USDT,1,binance


In [294]:
# data2 = data.drop("timestamp", axis=1)
sample_long = sample_long.drop_duplicates(
    subset=["timestamp", "exchange_id", "currency_pair", "level"]
)
sample_long = sample_long.drop('timestamp', axis=1)
sample_wide = cfibiask.transform_bid_ask_long_data_to_wide(sample_long, "timestamp")

In [295]:
sample_wide.head()
# sample_wide.to_csv("../data/transform/test/outcomes/SampleBidAskDataWide.csv")

,currency_pair,exchange_id,bid_price_l1,bid_price_l2,bid_size_l1,bid_size_l2,ask_price_l1,ask_price_l2,ask_size_l1,ask_size_l2
timestamp,,,,,,,,,,
2023-05-05 14:00:00.069000+00:00,ETH_USDT,binance,1931.65,1931.64,1.482,1.484,1931.67,1931.68,0.016,2.000
2023-05-05 14:00:00.094000+00:00,BTC_USDT,binance,29163.70,29163.60,8.581,0.004,29163.80,29163.90,1.762,0.177
2023-05-05 14:00:00.269000+00:00,ETH_USDT,binance,1931.97,1931.96,27.689,27.693,1931.98,1931.99,3.923,0.010
2023-05-05 14:00:00.294000+00:00,BTC_USDT,binance,29165.30,29165.20,5.552,4.331,29165.40,29165.50,2.276,0.533
2023-05-05 14:00:00.472000+00:00,ETH_USDT,binance,1932.19,1932.18,72.749,0.855,1932.20,1932.21,26.473,0.010
